<font color ='blue' size = '12'>**Осадки**</fon>

Данные с конкурса (https://www.kaggle.com/competitions/playground-series-s5e3/сode)

# <font color ='blue' size = '7'>**Импорт и установка библиотек**</font>

In [ ]:
from google.colab import drive
import json
import zipfile

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


In [ ]:
def load_dataset(from_kaggle:bool = False) -> pd.DataFrame:
  if from_kaggle:
    # запросит разрешение к гугл диску, необходимо дать это разрешение
    drive.mount('/content/drive')
    # установим kaggle
    !pip install kaggle -q
    !mkdir ~/.kaggle
    # копируем kaggle.json (предварительно, необходимо сгенерить токен на
    # сайте kaggle и сохранить к себе на гугл диск) в папку ~/.kaggle/
    !cp "/content/drive/MyDrive/Kaggle_API/kaggle.json" ~/.kaggle/
    !kaggle competitions download -c playground-series-s5e3
  else:
    !gdown 1-730JF1IWA5e_ejuXWLmkkzHFvudisdp
  # распаковка архива
  zip_ref = zipfile.ZipFile('playground-series-s5e3.zip', 'r')
  zip_ref.extractall()
  zip_ref.close()
  df_train = pd.read_csv('train.csv')
  df_test = pd.read_csv('test.csv')
  df_sample_submission = pd.read_csv('sample_submission.csv')
  return df_train, df_test, df_sample_submission

In [ ]:
df_train, df_test, df_sample_submission = load_dataset(from_kaggle=True)

##<font color = '5d98f0' size = '5'>На этих данных необходимо построить модель

In [ ]:
df_train.head(10)

In [ ]:
# # @title pressure vs maxtemp

# from matplotlib import pyplot as plt
# df_train.plot(kind='scatter', x='pressure', y='maxtemp', s=32, alpha=.8)
# plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
pd.set_option('display.max_columns', 30)

##<font color = '5d98f0' size ='5'>Определим роли для независимых признаков

In [ ]:
id = 'id' #  столбец ID
target = 'rainfall'  #  Целевая переменная

# <font color= 'blue' size = '7'>*EDA*</font>

##<font color = '5d98f0' size= '5'>Оценим размер dataset

In [ ]:
df_train.info()

In [ ]:
# Датасет занимает память 222.6 КВ и состоит из 13 столбцов 10 из котщрых с плавающей точкой

In [ ]:
df_train.shape

In [ ]:
df_train. corr()

In [ ]:
# Видим корреляцию dewpoint,maxtemp,temparature, mintemp

In [ ]:
# Рассмотрим основные характеристики
df_train.describe()

In [ ]:
# Проверим на дубликаты
df_train.duplicated()

In [ ]:
# Проверим на пропуски
df_train.isna().sum()

В наших данных нет пропусков и дубликатов

In [ ]:
# Посмотрим график по некторым столбцам
column = ['pressure', 'windspeed', 'cloud','dewpoint']
sns.pairplot(df_train[column])
plt.show()

In [ ]:
# Рассмотрим нашу целевую переменную на графике
df_train['rainfall'].value_counts().plot (kind='bar')

In [ ]:
#  Удалим не нужный столбец
df_train_1=df_train.drop('id', axis = 1)

In [ ]:
df_train_1.head(10)

#<font color = 'blue' size = '6'>**Разделим данные на test and train**</font>

In [ ]:
X = df_train_1.drop(['rainfall','day'], axis =1)
y = df_train_1['rainfall']
test = df_test.drop(columns=["id",'day'])

In [ ]:
# Подготовка тестового и тернировочного набора данных
X_train, X_test, y_train, y_train = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [ ]:
features_int = X.select_dtypes(include='float64').columns.to_list()
features_int

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[features_int], y, test_size=0.2, random_state=42)

#<font color= 'blue' size = '6'>**Построение моделей**</font>

#<font color = 'blue' size = '6'>**DecisionTreeClassifier**</font>

In [ ]:
pipeline_tree = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

In [ ]:
pipeline_tree.fit(X_train, y_train)

In [ ]:
y_pred = pipeline_tree.predict(X_test)
prediction = pipeline_tree.predict(X_test)
probability = pipeline_tree.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, prediction))
print("F1:", f1_score(y_test, prediction))
f1_clf = f1_score(y_test, prediction)

#<font color = 'blue' size= '6'>**RandomForest**</font>

In [ ]:
rf= RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
y_pred = pipeline_tree.predict(X_test)
prediction = pipeline_tree.predict(X_test)
probability = pipeline_tree.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, prediction))
print("F1:", f1_score(y_test, prediction))
f1_rf = f1_score(y_test, prediction)

##<font color ='5d98f0' size = '5'>Feature Selection

In [ ]:
feature_names= X.columns

In [ ]:
importances = rf.feature_importances_

In [ ]:
indices= np.argsort(importances)[::-1]

print('Feature importances:')
for f, idx in enumerate(indices):
  print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))

In [ ]:
best_features = indices[:10]
best_features_names = feature_names[best_features]
print(best_features_names)

In [ ]:
# d_first = 10
# plt.figure(figsize=(8, 8))
# plt.title("Feature importances")
# plt.bar(range(d_first), importances[indices[:d_first]], align='center')
# plt.xticks(range(d_first), np.array(feature_names)[indices[:d_first]], rotation=90)
# plt.xlim([-1, d_first]);

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#<font color ='blue' size = '6'>**GradientBoosting**</font>

In [ ]:
gbt= ensemble.GradientBoostingClassifier(n_estimators = 100, random_state= 11)

In [ ]:
# Построим модель со занчимыми признакми
gbt.fit(X_train[best_features_names], y_train)

In [ ]:
y_pred = gbt.predict(X_test[best_features_names])
prediction = gbt.predict(X_test[best_features_names])
probability = gbt.predict_proba(X_test[best_features_names])[:, 1]
print("Accuracy:", accuracy_score(y_test, prediction))
print("F1:", f1_score(y_test, prediction))
f1_gbt = f1_score(y_test, prediction)

#<font color = 'blue' size = '6'>Сonclusion

In [ ]:
pd.DataFrame(data = [f1_clf, f1_rf, f1_gbt], columns=["F1"], index = ["DecisionTreeClassifier", "RandomForestClassifier", "GradientBoostingClassifier"])


#<font color= 'blue' size = '6'>**Submission**</font>

In [ ]:
df_test.info()

In [ ]:
df_test.head(10)

In [ ]:
df_test.isna().sum()

In [ ]:
#  Числовой столбец с пропускаим у нас 1, заменим пропуски на среднее занчение
df_test['winddirection'].fillna(df_test['winddirection'].mean(), inplace=True)

In [ ]:
test.head()

In [ ]:
submit_score = []

In [ ]:
pred = gbt.predict_proba(test_scaled)[:,1]

In [ ]:
# submit= pd.read_csv('../input/playground-series-s5e3/sample_submission.csv')
#submission
df_sample_submission['rainfall'] = pred
df_sample_submission.to_csv('submission_svm.csv', index=False)
df_sample_submission.head()